# Not working
Some bugs with PEFT LORA model

In [ ]:
!pip install -q accelerate datasets peft bitsandbytes tensorboard trl wandb

In [ ]:
import os

from accelerate.utils import write_basic_config

write_basic_config()  # Write a config file
os._exit(00)  # Restart the notebook

In [1]:
# accelerate default config path
!cat ~/.cache/huggingface/accelerate/default_config.yaml

{
  "compute_environment": "LOCAL_MACHINE",
  "debug": false,
  "distributed_type": "MULTI_GPU",
  "downcast_bf16": false,
  "enable_cpu_affinity": false,
  "machine_rank": 0,
  "main_training_function": "main",
  "mixed_precision": "no",
  "num_machines": 1,
  "num_processes": 2,
  "rdzv_backend": "static",
  "same_network": false,
  "tpu_use_cluster": false,
  "tpu_use_sudo": false,
  "use_cpu": false
}


In [2]:
!nvidia-smi

Sat Feb  8 15:28:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P8             12W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Wandb

Creat token and account: https://wandb.ai/home

In [ ]:
import wandb
import getpass

token = getpass.getpass()
wandb.login(key=token)

In [3]:
from accelerate import PartialState
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

# Specify the checkpoint for SmolLM2 and set the device.
checkpoint = "HuggingFaceTB/SmolLM2-135M-Instruct"

# Load the tokenizer and model.
tokenizer = AutoTokenizer.from_pretrained(checkpoint,
                                          add_eos_token=True,
                                          add_bos_token=True)


def get_model():
    # For multi-GPU setups, consider using device_map="auto":
    model = AutoModelForCausalLM.from_pretrained(
            checkpoint,
            device_map={"": PartialState().process_index},
            attention_dropout=0.1,
            )
    return model

# Dataset

Json structure output: https://huggingface.co/datasets/ChristianAzinn/json-training

In [4]:
from datasets import load_dataset


def get_dataset():
    ds = load_dataset("ChristianAzinn/json-training")
    # Perform Train-Test Split
    split_ds = ds["train"].train_test_split(test_size=0.2, seed=42)

    # Access train and test splits
    train_dataset = split_ds["train"]
    test_dataset = split_ds["test"]
    return train_dataset, test_dataset

In [5]:
# Set the response template to match the chat format.
# (Ensure this string exactly matches the beginning of the assistant's response as output by apply_chat_template.)
response_template = "<|im_start|>assistant\n"
instruction_template = "<|im_start|>user\n"
PROMPT_TEMPLATE = """Query: {query}

schema:
{schema}"""


def formatting_prompts_func(example):
    """
    Converts each example into a conversation string using the tokenizer's chat template.
    Assumes each example contains lists under "instruction" and "output".
    """
    output_texts = []
    for i in range(len(example["query"])):
        # Build a conversation with a user message and an assistant reply.
        messages = [
            {
                "role":    "system",
                "content": "You are are an expert in generate json structure based on user query and schema."
                },
            {"role": "user", "content": PROMPT_TEMPLATE.format(query=example["query"][i], schema=example["schema"][i])},
            # Note: It is important that the assistant message content here does not
            # include the assistant marker, because the chat template will insert it.
            {"role": "assistant", "content": example["response"][i]}
            ]
        # Use the chat template to generate the formatted text.
        text = tokenizer.apply_chat_template(messages, tokenize=False)
        output_texts.append(text)
    return output_texts


def get_dataloarder():
    # Create the data collator.
    # It will search for the response_template (here "Assistant:") in the formatted text
    # and ensure that only tokens after that marker contribute to the loss.
    return DataCollatorForCompletionOnlyLM(response_template,
                                           instruction_template=instruction_template,
                                           tokenizer=tokenizer,
                                           mlm=False)

# SFT Trainer config

In [7]:
from peft import LoraConfig

OUTPUT_NAME = checkpoint.split("/")[-1] + "-structure-output"
OUTPUT_DIR = "/kaggle/working/" + OUTPUT_NAME


def get_trainer(model, train_dataset, test_dataset, data_collator):
    # Note that r, in the figure above, is a hyperparameter here that we can use to specify the rank of the low-rank matrices used for adaptation.
    # A smaller r leads to a simpler low-rank matrix, which results in fewer parameters to learn during adaptation.
    # This can lead to faster training and potentially reduced computational requirements.
    # However, with a smaller r, the capacity of the low-rank matrix to capture task-specific information decreases.
    # This may result in lower adaptation quality, and the model might not perform as well on the new task compared to a higher r.
    lora_config = LoraConfig(
            r=16,
            lora_alpha=32,
            lora_dropout=0.05,
            target_modules=['o_proj', 'k_proj', 'q_proj', "v_proj"],
            bias="none",
            task_type="CAUSAL_LM",
            )
    return SFTTrainer(
            model=model,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            args=SFTConfig(
                    per_device_train_batch_size=2,
                    gradient_accumulation_steps=4,
                    warmup_steps=100,
                    max_steps=1000,
                    learning_rate=0.0002,
                    lr_scheduler_type="cosine",
                    weight_decay=0.01,
                    bf16=True,
                    logging_strategy="steps",
                    logging_steps=10,
                    output_dir=OUTPUT_DIR,
                    optim="paged_adamw_8bit",
                    seed=42,
                    run_name=f"train-{OUTPUT_NAME}",
                    report_to="wandb",
                    save_steps=10,
                    save_total_limit=4,
                    ),
            peft_config=lora_config,
            formatting_func=formatting_prompts_func,
            data_collator=data_collator)

In [12]:
from transformers import is_torch_xpu_available, is_torch_npu_available
from peft import AutoPeftModelForCausalLM
from accelerate import Accelerator
import os


def main(mixed_precision="fp16", seed: int = 42, batch_size: int = 64):

    accelerator = Accelerator(mixed_precision=mixed_precision)
    model = get_model()
    model = accelerator.prepare(model)
    train_dataset, test_dataset = get_dataset()
    data_collator = get_dataloarder()
    trainer = get_trainer(model, train_dataset, test_dataset, data_collator)
    trainer.train()
    # Save fine tuned Lora Adaptor 
    trainer.model.save_pretrained(os.path.join(OUTPUT_DIR, "final_checkpoint"))
    # Free memory for merging weights
    del model
    if is_torch_xpu_available():
        torch.xpu.empty_cache()
    elif is_torch_npu_available():
        torch.npu.empty_cache()
    else:
        torch.cuda.empty_cache()

    model = AutoPeftModelForCausalLM.from_pretrained(OUTPUT_DIR, device_map="auto", torch_dtype=torch.bfloat16)
    model = model.merge_and_unload()

    output_merged_dir = os.path.join(OUTPUT_DIR, "final_merged_checkpoint")
    model.save_pretrained(output_merged_dir, safe_serialization=True)

In [9]:
from accelerate import notebook_launcher

In [13]:
args = ("fp16", 42, 64)
notebook_launcher(main, args, num_processes=2)

Launching training on 2 GPUs.


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:112: UserWarning: The pad_token_id and eos_token_id values of this tokenizer are identical. If you are planning for multi-turn training, it can result in the model continuously generating questions and answers without eos token. To avoid this, set the pad_token_id to a different value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:112: UserWarning: The pad_token_id and eos_token_id values of this tokenizer are identical. If you are planning for multi-turn training, it can result in the model continuously generating questions and answers without eos token. To avoid this, set the pad_token_id to a different value.
  warnings.warn(


Map:   0%|          | 0/16515 [00:00<?, ? examples/s]

W0208 15:37:39.148000 160 torch/multiprocessing/spawn.py:160] Terminating process 240 via signal SIGTERM
W0208 15:38:09.180000 160 torch/multiprocessing/spawn.py:168] Unable to shutdown process 240 via SIGTERM , forcefully exiting via SIGKILL
E0208 15:38:09.296000 160 torch/distributed/elastic/multiprocessing/api.py:732] failed (exitcode: 1) local_rank: 0 (pid: 238) of fn: main (start_method: fork)
E0208 15:38:09.296000 160 torch/distributed/elastic/multiprocessing/api.py:732] Traceback (most recent call last):
E0208 15:38:09.296000 160 torch/distributed/elastic/multiprocessing/api.py:732]   File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/api.py", line 687, in _poll
E0208 15:38:09.296000 160 torch/distributed/elastic/multiprocessing/api.py:732]     self._pc.join(-1)
E0208 15:38:09.296000 160 torch/distributed/elastic/multiprocessing/api.py:732]   File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/spawn.py", line 203, in join
E020

ChildFailedError: 
============================================================
main FAILED
------------------------------------------------------------
Failures:
  <NO_OTHER_FAILURES>
------------------------------------------------------------
Root Cause (first observed failure):
[0]:
  time      : 2025-02-08_15:37:38
  host      : 78f327634f51
  rank      : 0 (local_rank: 0)
  exitcode  : 1 (pid: 238)
  error_file: /tmp/torchelastic_ltj10a5p/none_z2bp4pjt/attempt_0/0/error.json
  traceback : Traceback (most recent call last):
    File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 355, in wrapper
      return f(*args, **kwargs)
    File "<ipython-input-12-da4fc2d8818d>", line 11, in main
      trainer = get_trainer(model, train_dataset, test_dataset, data_collator)
    File "<ipython-input-7-ea41278868f3>", line 20, in get_trainer
      return SFTTrainer(
    File "/usr/local/lib/python3.10/dist-packages/transformers/util

# inference

In [ ]:
import torch
import gc


def clear_hardwares():
    torch.clear_autocast_cache()
    torch.cuda.ipc_collect()
    torch.cuda.empty_cache()
    gc.collect()


clear_hardwares()
clear_hardwares()

## Lora adaptater

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained("SmolLM2-135M-Instruct-structure-output", return_dict=True,
                                                  device_map='auto', token='')
tokenizer = AutoTokenizer.from_pretrained(new_model, max_length=max_seq_length)
model = PeftModel.from_pretrained(base_model, new_model)
del base_model

## None lora

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"  # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR).to(device)

In [ ]:
test_json_schema = """{
  "type": "object",
  "properties": {
    "weather_data": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "year": { "type": "integer" },
          "station": { "type": "string" },
          "temperature": {
            "type": "object",
            "properties": {
              "min": { "type": "number" },
              "max": { "type": "number" }
            },
            "required": ["min", "max"]
          },
          "events": {
            "type": "array",
            "items": { "type": "string" }
          }
        },
        "required": ["year", "station", "temperature", "events"]
      }
    },
    "required": ["weather_data"]
  }
}"""

test_query = "Provide a detailed breakdown of meteorological data recorded in the city of Berlin from 2015 to 2020. The data should include the year, meteorological station, temperature ranges (minimum and maximum), and any significant events."

test_response = """{
  "weather_data": [
    {
      "year": 2015,
      "station": "Berlin Central Station",
      "temperature": { "min": -5.2, "max": 35.1 },
      "events": ["Heavy snowfall in January", "Heatwave in July"]
    },
    {
      "year": 2017,
      "station": "Berlin East Station",
      "temperature": { "min": -4.0, "max": 32.8 },
      "events": ["Thunderstorms in April", "Flooding in June"]
    },
    {
      "year": 2020,
      "station": "Berlin West Station",
      "temperature": { "min": -3.9, "max": 36.5 },
      "events": ["Drought in September", "Blizzards in February"]
    }
  ]
}"""

messages = [
    {
        "role":    "system",
        "content": "You are are an expert in generate json structure based on user query and schema."
        },
    {
        "role":    "user",
        "content": PROMPT_TEMPLATE.format(query=test_query, schema=test_json_schema)
        },
    ]

In [ ]:
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)
print("----------------- Generated text -----------------")
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=1024, temperature=0.2, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))